In [2]:
import requests
import json

# Replace with your JSONBin API key
API_KEY = "$2a$10$emdSh/thUg5xTvyY9z89UOy.BurV3ykvVjjUjodpvnJe2Xqc/WAAm"
BIN_API_URL = "https://api.jsonbin.io/v3/b"

# Load the JSON file
JSON_FILE = "questions.json"
with open(JSON_FILE, "r", encoding="utf-8") as file:
    data = json.load(file)

headers = {
    "Content-Type": "application/json",
    "X-Master-Key": API_KEY  # API key for authentication
}

response = requests.post(BIN_API_URL, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    bin_id = response.json()["metadata"]["id"]
    print(f"JSON uploaded successfully! Bin ID: {bin_id}")
    print(f"Access it at: https://api.jsonbin.io/v3/b/{bin_id}")
else:
    print(f"Upload failed: {response.text}")


JSON uploaded successfully! Bin ID: 67a87edbacd3cb34a8db4700
Access it at: https://api.jsonbin.io/v3/b/67a87edbacd3cb34a8db4700


In [1]:
import requests
import json

# Replace with your JSONBin API key
API_KEY = "$2a$10$emdSh/thUg5xTvyY9z89UOy.BurV3ykvVjjUjodpvnJe2Xqc/WAAm"
BIN_ID = "67a87edbacd3cb34a8db4700"
BIN_API_URL = f"https://api.jsonbin.io/v3/b/{BIN_ID}"

# JSON file path
JSON_FILE = "questions.json"

# Headers for authentication
headers = {
    "Content-Type": "application/json",
    "X-Master-Key": API_KEY
}

def upload_json():
    """Uploads the local JSON file to update the existing JSONBin."""
    try:
        with open(JSON_FILE, "r", encoding="utf-8") as file:
            data = json.load(file)

        response = requests.put(BIN_API_URL, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            print(f"JSON uploaded successfully! Bin ID: {BIN_ID}")
            print(f"Access it at: {BIN_API_URL}")
        else:
            print(f"Upload failed: {response.text}")

    except Exception as e:
        print(f"Error uploading JSON: {e}")

def download_json():
    """Downloads JSON from JSONBin and overwrites the local file."""
    try:
        response = requests.get(BIN_API_URL, headers=headers)

        if response.status_code == 200:
            data = response.json()["record"]
            with open(JSON_FILE, "w", encoding="utf-8") as file:
                json.dump(data, file, indent=4)
            print(f"JSON downloaded and saved to {JSON_FILE}")
        else:
            print(f"Download failed: {response.text}")

    except Exception as e:
        print(f"Error downloading JSON: {e}")

# Asking user for input
choice = input("Do you want to (U)pload or (D)ownload JSON? ").strip().lower()

if choice == "u":
    upload_json()
elif choice == "d":
    download_json()
else:
    print("Invalid choice! Please enter 'U' for upload or 'D' for download.")


Invalid choice! Please enter 'U' for upload or 'D' for download.


In [ ]:
import requests
import json
import os

class JSONManager:
    def __init__(self, api_key, bin_id, json_file):
        """Initialize API keys and file paths."""
        self.api_key = api_key
        self.bin_id = bin_id
        self.bin_api_url = f"https://api.jsonbin.io/v3/b/{bin_id}"
        self.json_file = json_file
        self.headers = {
            "Content-Type": "application/json",
            "X-Master-Key": api_key
        }

    def load_json(self):
        """Load the local JSON file."""
        try:
            with open(self.json_file, "r", encoding="utf-8") as file:
                return json.load(file)
        except (FileNotFoundError, json.JSONDecodeError):
            print(f"⚠️ Error: {self.json_file} not found or contains invalid JSON. Creating a new one.")
            return {"categories": {}}

    def save_json(self, data):
        """Save the JSON data locally."""
        with open(self.json_file, "w", encoding="utf-8") as file:
            json.dump(data, file, indent=4, ensure_ascii=False)
        print(f"✅ JSON file updated and saved locally.")

    def upload_json(self):
        """Upload the updated JSON file to JSONBin."""
        try:
            with open(self.json_file, "r", encoding="utf-8") as file:
                data = json.load(file)

            response = requests.put(self.bin_api_url, headers=self.headers, data=json.dumps(data))

            if response.status_code == 200:
                print(f"✅ JSON uploaded successfully! Bin ID: {self.bin_id}")
                print(f"📌 Access it at: {self.bin_api_url}")
            else:
                print(f"❌ Upload failed: {response.text}")

        except Exception as e:
            print(f"❌ Error uploading JSON: {e}")

    def create_category(self):
        """Create a new category."""
        data = self.load_json()
        new_category = input("Enter the new category name: ").strip()

        if new_category in data["categories"]:
            print(f"❌ Error: Category '{new_category}' already exists!")
            return

        data["categories"][new_category] = []
        self.save_json(data)
        print(f"✅ New category '{new_category}' created.")

        # Ask if user wants to add questions
        add_questions = input(f"Do you want to add questions to '{new_category}' now? (Y/N): ").strip().lower()
        if add_questions in ["y", "yes"]:
            self.append_questions(new_category)

    def append_questions(self, selected_category=None):
        """Append new questions to an existing category."""
        data = self.load_json()
        categories = list(data["categories"].keys())

        if not categories:
            print("❌ No categories available! Please create one first.")
            return

        # Choose category via number input
        if selected_category is None:
            print("\n📌 Existing Categories:")
            for idx, category in enumerate(categories, 1):
                print(f"{idx}) {category}")

            try:
                choice = int(input("\nSelect the category number to append questions: ").strip()) - 1
                if 0 <= choice < len(categories):
                    selected_category = categories[choice]
                else:
                    print("❌ Invalid selection.")
                    return
            except ValueError:
                print("❌ Please enter a valid number.")
                return

        # Choose input source
        source = input("Would you like to add questions from a (F)ile or (C)lipboard? ").strip().lower()
        new_questions = []

        if source == "f":
            file_path = input("Enter the JSON file path: ").strip()
            try:
                with open(file_path, "r", encoding="utf-8") as file:
                    new_questions = json.load(file)
            except Exception as e:
                print(f"❌ Error loading file: {e}")
                return
        elif source == "c":
            clipboard_data = input("Paste your JSON-formatted questions: ").strip()
            try:
                new_questions = json.loads(clipboard_data)
            except json.JSONDecodeError:
                print("❌ Error: Invalid JSON format.")
                return
        else:
            print("❌ Invalid choice. Please enter 'F' for file or 'C' for clipboard.")
            return

        if not self.validate_questions_format(new_questions):
            print("❌ Error: Invalid questions format. Ensure correct structure.")
            return

        data["categories"][selected_category].extend(new_questions)
        self.save_json(data)
        print(f"✅ Added {len(new_questions)} questions to '{selected_category}'.")

    def validate_questions_format(self, questions):
        """Check if the provided questions match the expected JSON structure."""
        if not isinstance(questions, list):
            return False
        for question in questions:
            if not all(key in question for key in ["question", "options", "correct_answer", "explanation"]):
                return False
            if not isinstance(question["options"], dict) or len(question["options"]) < 2:
                return False
        return True

    def main_menu(self):
        """Main interactive menu for JSON management."""
        while True:
            print("\n=== JSON Manager ===")
            print("1) Append questions to an existing category")
            print("2) Create a new category")
            print("3) Exit")

            choice = input("Select an option (1-3): ").strip()

            if choice == "1":
                self.append_questions()
            elif choice == "2":
                self.create_category()
            elif choice == "3":
                break
            else:
                print("❌ Invalid choice. Please select 1, 2, or 3.")

        # Ask whether to upload after changes
        upload_choice = input("Do you want to upload the updated JSON to JSONBin? (Y/N) [Default: Y]: ").strip().lower()
        if upload_choice in ["", "y"]:
            self.upload_json()
        else:
            print("🚀 Changes saved locally. Not uploaded.")

# Initialize JSONManager
if __name__ == "__main__":
    manager = JSONManager(
        api_key="$2a$10$emdSh/thUg5xTvyY9z89UOy.BurV3ykvVjjUjodpvnJe2Xqc/WAAm",
        bin_id="67a87edbacd3cb34a8db4700",
        json_file="questions.json"
    )
    manager.main_menu()



=== JSON Manager ===
1) Append questions to an existing category
2) Create a new category
3) Exit
❌ Invalid choice. Please select 1, 2, or 3.

=== JSON Manager ===
1) Append questions to an existing category
2) Create a new category
3) Exit
🚀 Changes saved locally. Not uploaded.


: 